In [3]:
import sys
sys.path.append('/home/shelunts/thesis/TabularSemantingParsing/')

In [4]:
from preprocess_inputs.parse_sql_one import get_schemas_from_json, Schema
from preprocess_inputs.process_sql import get_sql
from preprocess_inputs.get_tables import convert_fk_index, dump_db_json_schema
import json
import sqlite3
from os.path import isfile, isdir, join, split, exists, splitext
import pandas as pd
import nltk
import re
from src.eval.spider.evaluate import Evaluator
import pandas

In [5]:
ROOT_DIR = os.path.dirname(os.path.abspath(''))
DATA_DIR = ROOT_DIR + '/data/isrecon/isrecon/isrecon.sqlite'
SCHEMA_DIR = ROOT_DIR + '/data/spider/tables.json' 
SQL_DIR = ROOT_DIR + '/data/isrecon/isrecon/test_sql_pairs.sql'
DEV_DIR = ROOT_DIR + '/data/spider/dev.json'

In [29]:
PREDS_DIR_ACADEMIC = ROOT_DIR +'/data/spider/dev_academic.json'
LOG_DIR_ACADEMIC = ROOT_DIR + '/test_preds_outputs/inference_output2_academic.txt'
PREDS_DIR_COLLEGE = ROOT_DIR + '/data/spider/dev_college_2.json'
LOG_DIR_COLLEGE = ROOT_DIR + '/test_preds_outputs/inference_output8_on_college_placeholder.txt'
PREDS_DIR_SINGER = ROOT_DIR + '/data/spider/dev_singer.json'
LOG_DIR_SINGER = ROOT_DIR + '/test_preds_outputs/inference_output6_on_singer_placeholder.txt'
PREDS_DIR_CAR = ROOT_DIR + '/data/spider/dev_car_1.json'
LOG_DIR_CAR = ROOT_DIR + '/test_preds_outputs/inference_output9_on_cars_placeholder.txt'

Evaluation: Academic predictions

In [8]:
#initializing Evaluator to estimate the difficulty of SQL queries
eval = Evaluator()

In [9]:
#importing sqls and parsed sqls to estimate difficulty based on components
sqls_academic = json.load(open(PREDS_DIR_ACADEMIC))
queries_academic = [re.sub(";","",i["query"]) for i in sqls_academic]
parsed_queries_academic = [i['sql'] for i in sqls_academic]
hardness_status_academic = [eval.eval_hardness(i) for i in parsed_queries_academic]

In [12]:
#opening the file with wrong predictions as a look-up for mapping wrong predictions and doing some preprocessing
with open(LOG_DIR_ACADEMIC) as preds_file_academic:
    wrong_preds_output_academic = preds_file_academic.read().split('\n')
wrong_preds_academic = [re.sub("Target 0: b'",'',i) for i in wrong_preds_output_academic if len(re.findall('^Target 0: b', i))>0]
wrong_preds_academic = [re.sub("\'","",i) for i in wrong_preds_academic]
wrong_preds_academic = [re.sub("'","",i) for i in wrong_preds_academic]
pred_status_academic = [False if i in wrong_preds_academic else True for i in queries_academic]

In [13]:
for i in wrong_preds_academic[:5]:
    print(i)

SELECT DISTINCT t3.paperid FROM writes AS t3 JOIN author AS t2 ON t3.authorid  =  t2.authorid JOIN writes AS t4 ON t4.paperid  =  t3.paperid JOIN author AS t1 ON t4.authorid  =  t1.authorid WHERE t2.authorname  =  "Peter Mertens" AND t1.authorname  =  "Dina Barbian"
SELECT DISTINCT t3.paperid FROM writes AS t3 JOIN author AS t2 ON t3.authorid  =  t2.authorid JOIN writes AS t4 ON t4.paperid  =  t3.paperid JOIN author AS t1 ON t4.authorid  =  t1.authorid WHERE t2.authorname  =  "Peter Mertens" AND t1.authorname  =  "Dina Barbian"
SELECT DISTINCT t3.paperid FROM writes AS t3 JOIN author AS t2 ON t3.authorid  =  t2.authorid JOIN writes AS t4 ON t4.paperid  =  t3.paperid JOIN author AS t1 ON t4.authorid  =  t1.authorid WHERE t2.authorname  =  "Peter Mertens" AND t1.authorname  =  "Dina Barbian"
SELECT DISTINCT t3.paperid FROM writes AS t3 JOIN author AS t2 ON t3.authorid  =  t2.authorid JOIN writes AS t4 ON t4.paperid  =  t3.paperid JOIN author AS t1 ON t4.authorid  =  t1.authorid WHERE t2.

In [14]:
print("Distribution of right and wrong predictions\n")
pd.Series(pred_status_academic).value_counts(normalize=True)

Distribution of right and wrong predictions



False    0.973333
True     0.026667
dtype: float64

In [15]:
evaluation_df_academic = pd.DataFrame(data = {'SQL true': queries_academic, 'Hardness level': hardness_status_academic, 'Prediction status': pred_status_academic})
evaluation_df_academic.head()

,SQL true,Hardness level,Prediction status
0,SELECT DISTINCT t3.paperid FROM writes AS t3 J...,extra,False
1,SELECT DISTINCT t3.paperid FROM writes AS t3 J...,extra,False
2,SELECT DISTINCT t3.paperid FROM writes AS t3 J...,extra,False
3,SELECT DISTINCT t3.paperid FROM writes AS t3 J...,extra,False
4,SELECT DISTINCT t3.paperid FROM writes AS t3 J...,extra,False


In [16]:
print("Distribution of right and wrong predictions per SQL hardness groups\n")
pd.crosstab(index=evaluation_df_academic['Prediction status'], columns=evaluation_df_academic['Hardness level'],normalize='columns').round(3)*100

Distribution of right and wrong predictions per SQL hardness groups



Hardness level,easy,extra,hard,medium
Prediction status,,,,
False,62.1,99.8,99.3,93.7
True,37.9,0.2,0.7,6.3


Evaluation: College predictions

In [17]:
sqls = json.load(open(PREDS_DIR_COLLEGE))
queries_college = [re.sub(";","",i["query"]) for i in sqls]
parsed_queries_college = [i['sql'] for i in sqls]
hardness_status_college = [eval.eval_hardness(i) for i in parsed_queries_college]

In [18]:
with open(LOG_DIR_COLLEGE,'r') as preds_file_college:
    wrong_preds_output_college = preds_file_college.read().split('\n')
wrong_preds_college = [re.sub("^Target 0: b'",'',i) for i in wrong_preds_output_college if len(re.findall('^Target 0: b', i))>0]
wrong_preds_college = [re.findall("SELECT.+",i) for i in wrong_preds_college if len(re.findall("SELECT.+'",i))>0]
wrong_preds_college = [k for i in wrong_preds_college for k in i]
wrong_preds_college = [i.rstrip("'") for i in wrong_preds_college]
wrong_preds_college = [i.rstrip('"') for i in wrong_preds_college]
wrong_preds_college = [i.strip("'") for i in wrong_preds_college]
pred_status_college = [False if i in wrong_preds_college else True for i in queries_college]
#wrong_preds_college = [re.sub("42000'","42000",i) for i in wrong_preds_college]
#wrong_preds_college = [re.sub("teaches'","teaches",i) for i in wrong_preds_college]
evaluation_df_college = pd.DataFrame(data = {'SQL true': queries_college, 'Hardness level': hardness_status_college, 'Prediction status': pred_status_college})

In [19]:
for i in wrong_preds_college[:5]:
    print(i)

SELECT count(*) FROM classroom WHERE building != 'Lamberton
SELECT name FROM student WHERE dept_name  =  'History' ORDER BY tot_cred DESC LIMIT 1
SELECT name FROM student WHERE dept_name  =  'History' ORDER BY tot_cred DESC LIMIT 1
SELECT count(DISTINCT course_id) FROM course WHERE dept_name  =  'Physics
SELECT count(DISTINCT course_id) FROM course WHERE dept_name  =  'Physics


In [20]:
print("Distribution of right and wrong predictions per SQL hardness groups\n")
pd.crosstab(index=evaluation_df_college['Prediction status'], columns=evaluation_df_college['Hardness level'],normalize='columns').round(3)*100

Distribution of right and wrong predictions per SQL hardness groups



Hardness level,easy,extra,hard,medium
Prediction status,,,,
False,33.3,78.6,75.8,46.2
True,66.7,21.4,24.2,53.8


Evaluation: Singer predictions

In [21]:
sqls_singer = json.load(open(PREDS_DIR_SINGER))
queries_singer = [i["query"] for i in sqls_singer]
queries_singer = [re.sub(";","",i["query"]) for i in sqls_singer]
parsed_queries_singer = [i['sql'] for i in sqls_singer]
hardness_status_singer = [eval.eval_hardness(i) for i in parsed_queries_singer]

In [22]:
with open(LOG_DIR_SINGER) as preds_file_singer:
    wrong_preds_output_singer = preds_file_singer.read().split('\n')
wrong_preds_singer = [re.sub('^Target 0: b','',i) for i in wrong_preds_output_singer if len(re.findall('^Target 0: b', i))>0]
wrong_preds_singer = [i.rstrip("'") for i in wrong_preds_singer]
wrong_preds_singer = [i.strip('"') for i in wrong_preds_singer]
wrong_preds_singer = [i.lstrip("'") for i in wrong_preds_singer]
pred_status_singer = [False if i in wrong_preds_singer else True for i in queries_singer]
evaluation_df_singer = pd.DataFrame(data = {'SQL true': queries_singer, 'Hardness level': hardness_status_singer, 'Prediction status': pred_status_singer})

In [23]:
for i in wrong_preds_singer[:5]:
    print(i)

SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
select max(capacity), average from stadium
SELECT name ,  capacity FROM stadium ORDER BY average DESC LIMIT 1
select count(*) from concert where stadium_id = (select stadium_id from stadium order by capacity desc limit 1)
select count(*) from concert where stadium_id = (select stadium_id from stadium order by capacity desc limit 1)


In [24]:
pd.Series(hardness_status_singer).value_counts(normalize=True).round(2)

medium    0.53
hard      0.29
easy      0.09
extra     0.09
dtype: float64

In [25]:
print("Distribution of right and wrong predictions per SQL hardness groups\n")
pd.crosstab(index=evaluation_df_singer['Prediction status'], columns=evaluation_df_singer['Hardness level'],normalize='columns').round(3)*100

Distribution of right and wrong predictions per SQL hardness groups



Hardness level,easy,extra,hard,medium
Prediction status,,,,
False,0.0,0.0,15.4,20.8
True,100.0,100.0,84.6,79.2


Evaluation: Car predictions

In [26]:
sqls = json.load(open(PREDS_DIR_CAR))
queries_cars = [re.sub(";","",i["query"]) for i in sqls]
parsed_queries_cars = [i['sql'] for i in sqls]
hardness_status_cars = [eval.eval_hardness(i) for i in parsed_queries_cars]

In [27]:
pd.Series(hardness_status_cars).value_counts(normalize=True).round(2)

medium    0.35
extra     0.28
easy      0.20
hard      0.17
dtype: float64

In [30]:
with open(LOG_DIR_CAR) as preds_file_cars:
    wrong_preds_output_cars = preds_file_cars.read().split('\n')
wrong_preds_cars = [re.sub('^Target 0: b','',i) for i in wrong_preds_output_cars if len(re.findall('^Target 0: b', i))>0]
wrong_preds_cars = [i.replace("\\", "") for i in wrong_preds_cars]
wrong_preds_cars = [i.rstrip("'") for i in wrong_preds_cars]
wrong_preds_cars = [i.strip('"') for i in wrong_preds_cars]
wrong_preds_cars = [i.lstrip("'") for i in wrong_preds_cars]
pred_status_cars = [False if i in wrong_preds_cars else True for i in queries_cars]
#wrong_preds_college = [re.sub("42000'","42000",i) for i in wrong_preds_college]
#wrong_preds_college = [re.sub("teaches'","teaches",i) for i in wrong_preds_college]
evaluation_df_cars = pd.DataFrame(data = {'SQL true': queries_cars, 'Hardness level': hardness_status_cars, 'Prediction status': pred_status_cars})

In [31]:
for i in wrong_preds_cars[:5]:
    print(i)

SELECT T1.ContId ,  T1.Continent ,  count(*) FROM CONTINENTS AS T1 JOIN COUNTRIES AS T2 ON T1.ContId  =  T2.Continent GROUP BY T1.ContId
SELECT T1.ContId ,  T1.Continent ,  count(*) FROM CONTINENTS AS T1 JOIN COUNTRIES AS T2 ON T1.ContId  =  T2.Continent GROUP BY T1.ContId
SELECT T1.FullName ,  T1.Id ,  count(*) FROM CAR_MAKERS AS T1 JOIN MODEL_LIST AS T2 ON T1.Id  =  T2.Maker GROUP BY T1.Id
SELECT T1.Model FROM CAR_NAMES AS T1 JOIN CARS_DATA AS T2 ON T1.MakeId  =  T2.Id ORDER BY T2.horsepower ASC LIMIT 1
SELECT T1.Model FROM CAR_NAMES AS T1 JOIN CARS_DATA AS T2 ON T1.MakeId  =  T2.Id ORDER BY T2.horsepower ASC LIMIT 1


In [32]:
pd.Series(hardness_status_cars).value_counts(normalize=True).round(2)

medium    0.35
extra     0.28
easy      0.20
hard      0.17
dtype: float64

In [33]:
evaluation_df_cars['Prediction status'].value_counts(normalize=True).round(2)

False    0.74
True     0.26
Name: Prediction status, dtype: float64